In [5]:
from tqdm import tqdm
import random as rand
import numpy as np
import time
import copy

# Plotting Packages
from IPython import display
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, AutoMinorLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [6]:
class MDP_Environment():
    def __init__(self, rewards = None):
        
        # Defining the number of states & actions
        self.num_states  = 4
        self.num_actions = 2
        
        # Defining the rewards action dynamic 
        self.rewards = np.array([[3, 2, 1, 0],[2,1,3,10]]) 

        # Definining the Initial State Value function
        self.S = np.zeros(self.num_states)

        # Definining the Initial Equi-probable Random policy
        # The first index represents the state
        # The sexond index represents the action in any particular state
        self.pi = np.ones((4, 2)) * 0.5

        # DEBUG CODE | IGNORE
        # self.S = np.arange(16).reshape((4, 4))
        # self.pi = np.zeros((4, 4, 4))
        # self.pi[0][0] = [1, 0, 0, 0]
        # self.pi[0][3] = [0, 0, 0, 1]
        # self.pi[2][2] = [0, 1, 0, 0]
        # self.pi[3][0] = [0, 0, 1, 0]

    
        # Defining the transition probabilities {p(s'|s, a)}
        temp_0 = np.array([
            [1, 0, 0, 0],
            [0.2, 0.8, 0, 0],
            [0, 0.2, 0.8, 0],
            [0, 0, 0.2, 0.8]
        ])

        temp_1 = np.array([
            [0, 0.2, 0.3, 0.5],
            [0.5, 0, 0.5, 0],
            [0, 0.5, 0, 0.5],
            [0, 0.5, 0.5, 0]
        ])
        self.tran_probs = np.ones((2, 4, 4))
        self.tran_probs[0, :, :] = temp_0
        self.tran_probs[1, :, :] = temp_1


In [7]:
def policy_iteration(dis_factor, theta, max_iters, rewards = None):
    env = MDP_Environment(rewards)
    policy_stable = False
    iters = 1

    while not policy_stable and iters <= max_iters:
        print(f"\nPolicy Iteration  | Iteration = {iters}")

        # Policy Evaluation (or Prediction)
        evaluate_policy(env, dis_factor, theta)
        
        print(np.round(env.S, 2))
        # Policy Improvement
        policy_stable = improve_policy(env, dis_factor)

        iters += 1

    # DEBUG CODE | IGNORE
    print("Optimal State Value Function:")
    print(np.round(env.S, 2))
    print("Optimal Policy:")
    print(env.pi)

    # Plot the optimal value function and optimal policy, once the algo has converged
    #env.plot_states()
    #env.plot_policy()

    
#state value for a given policy
def evaluate_policy(env, dis_factor, theta):
    n = 0
    temp_delta = copy.copy(theta)

    s_value=np.zeros(env.num_states)

    while temp_delta >= theta :
        temp_delta = 0
        prev_s_value=s_value
        for s in range(env.num_states):
            temp_stval=prev_s_value[s]
            s_value[s]=0
            
            for a in range(env.num_actions):
        
                for s_prime in range(env.num_states):
                    s_value[s] +=env.pi[s][a] * env.tran_probs[a, s, s_prime] * (
                        env.rewards[a][s] + dis_factor * prev_s_value[s_prime]
                    )


            temp_delta = max(temp_delta, abs(temp_stval- s_value[s]))

        n += 1
    
    env.S=s_value

def improve_policy(env, dis_factor):
    print(f"Policy Improvement")
    policy_stable = True
    
    old_pi=copy.copy(env.pi)
    for s in range(env.num_states):
        
        max_val = 1e-5
        for a in range(env.num_actions):
            s_value=0
            for s_prime in range(env.num_states):
                s_value +=env.pi[s][a] * env.tran_probs[a, s, s_prime] * (
                    env.rewards[a][s] + dis_factor * env.S[s_prime]
                )

            if(s_value>max_val):
                max_val=s_value
                a_star=a
        
        #updating policy
        for a in range(env.num_actions):
            if a==a_star :
                env.pi[s][a]=1
                env.pi[s][1-a]=0
        

    if not np.array_equal(old_pi, env.pi):
        policy_stable =  False

    return policy_stable


In [4]:
## Initializing the variables
# Discount Factor
dis_factor = 0.9
# Threshold for stopping the policy evaluation (or prediction) algo
theta = 1e-2

# Maximum Iterations for the Policy Iteration Algorithms
max_iters = 20

# Using the original rewards distribution
policy_iteration(dis_factor, theta, max_iters)



Policy Iteration  | Iteration = 1
[5.45 4.69 5.45 7.95]
Policy Improvement

Policy Iteration  | Iteration = 2
[ 3.    3.22 12.04 16.86]
Policy Improvement
Optimal State Value Function:
[ 3.    3.22 12.04 16.86]
Optimal Policy:
[[1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]


[[3, 4, 5], [1, 8, 9]]
